In [2]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa

def reshape_images(images, height, width):
    reshaped_images = np.zeros((len(images), height, width, 3))
    for n in range(len(images)):
        for h in range(height):
            for w in range(width):
                reshaped_images[n, h, w, images[n][h][w]] = 1
    return reshaped_images

def augment_images(images, number=None):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
            rotate=(-180, 180),
            shear=(-8, 8)
        ),
        iaa.GaussianBlur(sigma=(0.0, 3.0)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),
        iaa.Multiply((0.8, 1.2), per_channel=0.2)
    ], random_order=True)
    
    images_input = np.random.choice(images, number) if number else images
    images_expanded = []
    for image in images_input:
        images_expanded.append(image)
    images_expanded = np.array(images_expanded)
    images_augmented = seq(images=images_expanded)
    return images_augmented



In [3]:
class_num = 9
dsize = (224, 224)
count_per_class_test = 20
count_per_class = 10000  # Adjusted count_per_class for training

data_directory = "Images"  # Change this to the path of your local directory
outPut='NewAugmentation'
x_train, y_train = [], []

for folder in os.listdir(data_directory):
    i=0
    output_foler=os.path.join(outPut,folder)
    os.makedirs(output_foler, exist_ok=True) 
    images_path = os.path.join(data_directory, folder)
    images_list = os.listdir(images_path)
    
    for img_filename in np.random.choice(images_list, count_per_class, replace=True):
        img_path = os.path.join(images_path, img_filename)
        img = cv2.imread(img_path)
        resized = cv2.resize(img, dsize=dsize, interpolation=cv2.INTER_AREA)
        del img
        augmented = np.array(augment_images([resized]))
        reshaped = reshape_images(augmented, dsize[1], dsize[0])
        del augmented
        i+=1
        img_path=os.path.join(output_foler,f"{folder}_{i}.jpeg")
        cv2.imwrite(img_path)
#         labels = np.zeros((1, class_num))
#         labels[0][failureNum] = 1

#         x_train.extend(reshaped)
#         y_train.extend(labels)

x_train = np.array(x_train)
y_train = np.array(y_train)

IndexError: index 15 is out of bounds for axis 3 with size 3

In [5]:
import os
import cv2
import numpy as np
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import imgaug.augmenters as iaa

def reshape_images(images, height, width):
    reshaped_images = np.zeros((len(images), height, width, 3))
    for n in range(len(images)):
        for h in range(height):
            for w in range(width):
                reshaped_images[n, h, w, images[n][h][w]] = 1
    return reshaped_images

def augment_images(images, target_count):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
            rotate=(-180, 180),
            shear=(-8, 8)
        ),
        iaa.GaussianBlur(sigma=(0.0, 3.0)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),
        iaa.Multiply((0.8, 1.2), per_channel=0.2)
    ], random_order=True)
    
    current_count = len(images)
    needed_augmentations = max(0, target_count - current_count)
    
    if needed_augmentations > 0:
        images_input = np.random.choice(images, needed_augmentations) if needed_augmentations < current_count else images
        images_expanded = [image for image in images_input]
        images_expanded = np.array(images_expanded)
        images_augmented = seq(images=images_expanded)
        return np.concatenate([images, images_augmented], axis=0)
    else:
        return images

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

def save_images_to_folder(images, folder):
    os.makedirs(folder,exist_ok=True)
    for i, image in enumerate(images):
        cv2.imwrite(os.path.join(folder, f"augmented_{i}.png"), image)

def main():
    main_folder = "Images"  # Update this with the actual path to your main folder
    subfolders = os.listdir(main_folder)
    target="NewAugmentation"
    for subfolder in subfolders:
        subfolder_path = os.path.join(main_folder, subfolder)
        images = load_images_from_folder(subfolder_path)

        # Reshape images
        reshaped_images = reshape_images(images, height, width)

        # Augment images to reach a total of 10000
        augmented_images = augment_images(reshaped_images, target_count=10000)

        # Save augmented images to the same folder
        tar=os.path.join(target,subfolder)
        save_images_to_folder(augmented_images, subfolder_path)



In [6]:
if __name__ == "__main__":
    height = 224  # Update with the desired height
    width = 224   # Update with the desired width
    main()


IndexError: index 115 is out of bounds for axis 3 with size 3